## Scraping Job Posting Data from Indeed 
Source:
https://towardsdatascience.com/scraping-job-posting-data-from-indeed-using-selenium-and-beautifulsoup-dfc86230baac

strategy:

* Get all the job posting links
* Click each link and parse text from the job posting page 
* Store the parsed text data

The questions to address
* optimizing query strategy

Researching job analysis with machine learning

https://www.jobspikr.com/blog/analysis-of-machine-learning-job-listings-data-reveals-the-key-skills/

 Further, we find that data science jobs can be grouped into three main personas: Core data scientists, researchers, and big data specialists. https://www.glassdoor.com/research/data-scientist-personas/
 


### 1. Get all the job posting links

In [63]:
# all import statements

from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
import re
from collections import defaultdict

# to write from list to csv file
import csv

In [2]:
 """
    Given the url of a page, this function returns the soup object.
    
    Parameters:
        url: the link to get soup object for
    
    Returns:
        soup: soup object
    """

def get_soup(url):
    driver = webdriver.Chrome("C:/Users/lili/Documents/icode/scraping/chromedriver")
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    driver.close()
    return soup
#soup = get_soup("https://il.indeed.com/jobs?q=data+scientist&l=tel+aviv&start=10")


In [3]:
   """
    Grab all non-sponsored job posting links from a Indeed search result page using the given soup object
    
    Parameters:
        soup: the soup object corresponding to a search result page
                e.g. https://ca.indeed.com/jobs?q=data+scientist&l=Toronto&start=20
    
    Returns:
        urls: a python list of job posting urls
    
    """
def grab_job_links(soup):
    urls = []
    for link in soup.find_all('h2', {'class': 'jobtitle'}):
        partial_url = link.a.get('href')
        url = 'https://il.indeed.com' + partial_url
        urls.append(url)
    return urls

#urls = grab_job_links(soup)


In [4]:
 """
    Grab number of result pages, from a Indeed search result page using the given soup object
    
    Parameters:
        soup: the soup object corresponding to a search result page
                e.g. https://ca.indeed.com/jobs?q=data+scientist&l=Toronto&start=20
    
    Returns:
        num_pages: integer number of result pages
    
    """
def grab_num_pages(soup):
    page_in_search = soup.find(name='div', attrs={'id':"searchCount"}).get_text()
    p = re.compile('\d+')
    num_pages = p.findall(page_in_search)
    num_pages = int(num_pages[1])
    return num_pages


In [40]:
"""
    Get all the job posting URLs resulted from a specific search.
    
    Parameters:
        query: job title to query
        num_pages: number of pages needed
        location: city to search in
    
    Returns:
        urls: a list of job posting URL's (when num_pages valid)
        max_pages: maximum number of pages allowed ((when num_pages invalid))
    """
query = ["data+scientist"]
location = ["Tel+Aviv"]

def get_urls(query, num_pages, location):
    # We always need the first page
    base_url = 'https://il.indeed.com/jobs?q={}&l={}'.format(query, location)
    soup = get_soup(base_url)
    urls = grab_job_links(soup)
    num_listings = grab_num_pages(soup)
    print("num_listings",num_listings)
    num_pages_calc = int(num_listings/10)
    print("num_pages_calc",num_pages_calc)

    # starting page 2
    for i in range(2, num_pages_calc+1):
            num = (i-1) * 10
            base_url = 'https://il.indeed.com/jobs?q={}&l={}&start={}'.format(query, location, num)
            try:
                soup = get_soup(base_url)
                # We always combine the results back to the list
                urls += grab_job_links(soup)
            except:
                continue
    return urls 
    
# finished stage 1 we have urls a list  with job posting links

In [41]:
# just a check that every thing works so far

urls = get_urls(query, 5, location)
print(len(urls))
print(urls)

num_listings 174
num_pages_calc 17
163
['https://il.indeed.com/rc/clk?jk=f1ef24688a24a55b&fccid=c869809b954e0ce1&vjs=3', 'https://il.indeed.com/rc/clk?jk=4f52ebcb098084d5&fccid=978d9fd9799d55a8&vjs=3', 'https://il.indeed.com/company/BigaBid/jobs/Data-Scientist-29ca8238428aa6ba?fccid=620c28d86cfedd30&vjs=3', 'https://il.indeed.com/rc/clk?jk=2d6732ccf0f8278a&fccid=1c6aeebf4483d9bf&vjs=3', 'https://il.indeed.com/rc/clk?jk=c701e1452fda60aa&fccid=cad845b9fe80e774&vjs=3', 'https://il.indeed.com/rc/clk?jk=53915167e4807347&fccid=59b0d308aef1d946&vjs=3', 'https://il.indeed.com/rc/clk?jk=69df6a98bcc1a484&fccid=502732c3841cad8b&vjs=3', 'https://il.indeed.com/rc/clk?jk=b84afe72af7b3aca&fccid=502732c3841cad8b&vjs=3', 'https://il.indeed.com/company/BIYOND,-AWS-PARTNER/jobs/Data-Scientist-34600ddffe510397?fccid=5a712464d4a73a07&vjs=3', 'https://il.indeed.com/rc/clk?jk=a08e09c51ed76d8b&fccid=bd3717e9a69d0bd3&vjs=3', 'https://il.indeed.com/rc/clk?jk=f2d6ed3c0f2a72df&fccid=1c6aeebf4483d9bf&vjs=3', 'http

### 2. Click each link and parse text from the job posting page

In [17]:
"""
    Get the text portion including both title and job description of the job posting from a given url
    
    Parameters:
        url: The job posting link
        
    Returns:
        title: the job title (if "data scientist" is in the title)
        posting: the job posting content    
    """

def get_posting1(url):
    soup = get_soup(url)
    title = soup.find(name='h3').getText().lower()
    posting = soup.find(name='div', attrs={'class': "jobsearch-JobComponent"}).get_text()
    return title, posting.lower()
    

In [42]:
'''
    get title + discribtion for every job posting url
    
    parameters: 
            url list 'urls'
            
    returns: 
            list with tuples job title and description
'''

posting_list =[]
for i in range(0, len(urls)):
    posting = get_posting1(urls[i])
    posting_list.append(posting)
  

i is https://il.indeed.com/rc/clk?jk=f1ef24688a24a55b&fccid=c869809b954e0ce1&vjs=3
title is data scientist
posting is Data ScientistTaboola-תל אביב -יפו, מחוז תל אביבApply NowApply On Company SiteSave this jobTaboola, the world’s largest content recommendation platform, is hiring a Data Scientist.

In this job you will:

Be responsible for the entire algorithmic life-cycle in the company: data analytics, prototyping of new ideas, implementing algorithms in a production environment and then monitoring and maintaining them
Turn algorithm prototypes into shippable products that will have a significant and immediate impact on the company’s revenue
Work on a daily basis with some of the hottest trends in today’s job market: machine/deep learning, big data analytics and cloud computing
Apply your scientific knowledge and creativity to analyze large volumes of diverse data and develop algorithms to solve complex problems
Influence directly on the way billions of people discover the internet
W

i is https://il.indeed.com/rc/clk?jk=2d6732ccf0f8278a&fccid=1c6aeebf4483d9bf&vjs=3
title is data scientist
posting is Data ScientistIntango-תל אביב -יפו, מחוז תל אביבApply NowApply On Company SiteSave this jobWe’re looking for a brilliant **Data Scientist** to join our CTO team
The CTO team is developing data solutions to serve Intago’s current and future products.

In our day to day, we’re collecting enormous amount of data and design the big data workflow: pipeline, storage and analytics, in order to make smart decisions offline and on-the-fly. We are pragmatic and love not just to research, but also to execute and bring our ideas into life.

The team is a team of specialists, each one of us is leading an independent work stream but is also a team member and share knowledge.

We love technologies and data and looking for somebody that shares our passion to build solutions that carries tons of data!

You will:

Develop new data models and strategies
Research, develop, test and evaluat

i is https://il.indeed.com/company/BIYOND,-AWS-PARTNER/jobs/Data-Scientist-34600ddffe510397?fccid=5a712464d4a73a07&vjs=3
title is data scientist
posting is Data ScientistBIYOND, AWS PARTNER-תל אביב -יפו, מחוז תל אביבResponded to 51-74% of applications in the past 30 days, typically within 1 day.Apply NowApply NowSave this jobBIyond is looking for a Data Scientist to join our Data Analytics experts team.If you’re an experienced Data Scientist with solid statistical and/or mathematical background, and a huge passion for ML/AI – you’re going to love this opportunity!Job requirements3-5 years’ experience working as a data scientist.Highly proficient in Python.Ability to design and implement Advanced Analytics solutions end-to-endsc. or higher in Statistics, Computer Science and/or MathematicsPassion for Data & AnalyticsSmart, problem solverTeam player with a can-do approach.AdvantageExperience with Big Data technologies, e.g. Hadoop, SparkExperience with ML/AI solutions in AWS and/or Azure

i is https://il.indeed.com/rc/clk?jk=c615fe8e8dcb7a1f&fccid=8e382eee9bb3544b&vjs=3
title is data scientist לחברת סקיוריטי עבור הרכבים האוטונומים
posting is Data Scientist לחברת סקיוריטי עבור הרכבים האוטונומיםSQlink-תל אביב -יפו, מחוז תל אביבApply NowApply On Company SiteSave this jobתיאור המשרה:
לחברת סטארטאפ של אחד היזמים הבולטים כיום בתעשית הסיבר דרוש/ה Data Scientist שמעוניין/ת להיות חלק מצוות פיתוח קטן בחזית הטכנולוגיה עבור הדור הבא של פלטפורומות סקיוריטי עבור יצרני רכב אוטונומי.

לחברה שיתופי פעולה אסטרטגים עם יצרניות רכב מובילות. סיימו סבב גיוס מוצלח מאוד. הזדמנות מעולה להצטרף לעבודה על מוצר חדש שעתיד לשנות את עולם הרכב האוטונומי!
דרישות המשרה:

10 שנות ניסיון כמהנדס/ת אלגוריתמים
-ניסיון ב-Python ו- R הכרחי
ניסיון בפיתוח ב-C יתרון משמעותיBig Data יתרון
מס' משרה: 98870SQLink - 15 days ago - save job - report job - original job
i is https://il.indeed.com/rc/clk?jk=3cd481c98e753b79&fccid=8e382eee9bb3544b&vjs=3
title is data scientist
posting is Data ScientistSQlink-תל אביב -יפו, מחו

i is https://il.indeed.com/rc/clk?jk=cd8c1bd76f198a01&fccid=0ab947bbb49672a1&vjs=3
title is data scientist
posting is Data ScientistViaVan-תל אביב -יפו, מחוז תל אביבApply NowApply On Company SiteSave this jobWe are Via – Ride with us!

Via is developing the operating system that manages and controls shared mobility at scale.
Our platform provides many millions of rides to consumers in NYC, Chicago, Washington DC, London, Berlin, Amsterdam and many more locations.
We are helping cities transition their bus systems from rigid lines and fixed schedules to on-demand networks of dynamic shuttles.
Our Tel-Aviv office is full of passionate and innovative professionals who love working together to make something great. If you are up for a challenge and have a true desire to make an impact - Come ride with us!

The Position:

We are looking for a Data Scientist to join our R&D center in TLV.

The position is to be part of a Development team responsible for Via’s research in wide variety of topi

...

i is https://il.indeed.com/rc/clk?jk=9303e436ed79cafd&fccid=08487f979320ebae&vjs=3
title is quantitative researcher
posting is Quantitative ResearcherWorldQuant-רמת גן, מחוז תל אביבApply NowApply On Company SiteSave this jobWorldQuant develops and deploys systematic financial strategies across a variety of asset classes and global markets. We seek to produce high-quality predictive signals (Alphas) through our proprietary research platform to employ financial strategies focused on exploiting market inefficiencies. Our teams work collaboratively to drive the production of Alphas and financial strategies – the foundation of a sustainable, global investment platform.
WorldQuant’s success is built on a culture that pairs academic sensibility with accountability for results. Employees are encouraged to think openly about problems, balancing intellectualism and practicality. Great ideas come from anyone, anywhere. Employees are encouraged to challenge conventional thinking and possess a mind

i is https://il.indeed.com/rc/clk?jk=fe41859ca3283219&fccid=c1ee211a89cc3f56&vjs=3
title is director, cyber security protection
posting is Director, Cyber Security ProtectionCheck Point Software Technologies Ltd.-תל אביב -יפו, מחוז תל אביבApply NowApply On Company SiteSave this jobBe the leader of Check Point Cyber security products on the network and cloud, be the leader of successful, innovative products with unique and accurate contribution to our customers and partners.

 Experienced VP Level RND software developer manager/director - experienced in managing software development groups (50+ software developers). In this position you will manage 70+ developers, data scientist, project managers and product managers
Innovative and dynamic thinker who can bring new Ideas to execution.
Understand customers’ needs and ability to drive product vision. Roadmap planning and execution
Understand the cyber security market technically and sales wise.
Transition the area to be fully agile releas

i is https://il.indeed.com/rc/clk?jk=a01fa2eeb46697cf&fccid=7bf7fbdb7ede703e&vjs=3
title is data scientist team leader
posting is Data scientist Team leaderPeopleCom Solutions LTD-תל אביב -יפו, מחוז תל אביבApply NowApply On Company SiteSave this jobלחברת סטארטאפ מובילה ומבוססת הממוקמת בת"א, דרוש/ה Data scientist Team leader. - תואר מתקדם בתחום/ ניסיון מקביל - 5 שנות ניסיון ומעלה כ- Data Scientist ובעבודה עם Machine Learning - לפחות שנתיים ניסיון ניהולי בתחום - ניסיון בעבודה עם אלגוריתמים וב- PythonJobnet Click & Work - 30+ days ago - save job - report job - original job
i is https://il.indeed.com/rc/clk?jk=f696aa3face3545f&fccid=52b211fcc6210777&vjs=3
title is machine learning expert
posting is Machine Learning ExpertVonage R&D Center Israel-תל אביב -יפו, מחוז תל אביבApply NowApply On Company SiteSave this jobVonage’s R&D Center is building a new machine learning and data science team and is looking for individuals skilled in both research and code to take us to the next level.
As a Vo

i is https://il.indeed.com/rc/clk?jk=8d763526e26dadef&fccid=de2a910cf95906e3&vjs=3
title is security engineer
posting is Security engineerCellebrite-פתח תקווה, מחוז המרכזApply NowApply NowSave this jobCellebrite is looking for a Security Engineer to join the security team.

Responsibilities & Scope of RoleTake part in Developing and integrating the Company’s data security policiesEvaluate various security solutions.Install and configure various security solutionsTroubleshoot and support security related issues.Wiling to support 24x7 per need
Requirements:
Qualifications & ExperienceBachelor’s degree- preferred computer scientist or SW engineering.CCNA and relevant security certification- Advantage.3-4 years’ experience in cyber security from a global company.Vast experience with the following solutions: McAfee Endpoint protection, Forcepoint Proxy, DLP, Cloud email, Checkpoint FW, IBM Qradar SIEM, File system security and permissions, GPO, NAC, NBAD, Incident response.Vast experience w

i is https://il.indeed.com/rc/clk?jk=8a80e83d9db0110d&fccid=6223ded47d6e8d79&vjs=3
title is software development team leader
posting is Software Development Team LeaderSamsung NEXT-תל אביב -יפו, מחוז תל אביבApply NowApply On Company SiteSave this jobAbout MissingLink.ai
At MissingLink.ai we’re at the forefront of technology. In the next 3 years, every company is going to build an AI machine to implement a capability that is impossible to express using code like we’re used to. With MissingLink we’re allowing every engineer and Data Scientist to build complex AI machines in a way that was not possible before.
We provide a set of lifecycle management tools for Deep Learning and Machine Learning, we automatically track experiments, data, machines, models versions and visualize the learning process in real time in order to help make decisions faster and drive insights on what to do next to improve the model.
Customers we work with today solve a wide range of problems like fraud detection, I

In [43]:
# check len posting list 

print(len(posting_list))
print(posting_list)


163
[('data scientist', 'data scientisttaboola-תל אביב -יפו, מחוז תל אביבapply nowapply on company sitesave this jobtaboola, the world’s largest content recommendation platform, is hiring a data scientist.\n\nin this job you will:\n\nbe responsible for the entire algorithmic life-cycle in the company: data analytics, prototyping of new ideas, implementing algorithms in a production environment and then monitoring and maintaining them\nturn algorithm prototypes into shippable products that will have a significant and immediate impact on the company’s revenue\nwork on a daily basis with some of the hottest trends in today’s job market: machine/deep learning, big data analytics and cloud computing\napply your scientific knowledge and creativity to analyze large volumes of diverse data and develop algorithms to solve complex problems\ninfluence directly on the way billions of people discover the internet\nwork on projects such as internet personalization, content feed, real time bidding, v

In [84]:
# take posting list and count relevant vs. irrelevant

data_scientist_jobs = defaultdict(list)
#data_analyst_jobs = defaultdict(list)
data_sci_des = []
data_eng_des = []
data_ana_des = []
datasci_related = [] 
non_data_sc = []


for k in posting_list:
    if 'data' in k[0] and (('scientist' in k[0]) or ('science' in k[0])):
        data_sci_des.append(k) 
        if k[0] in data_scientist_jobs.keys():
            data_scientist_jobs[k[0]] += 1
        else:
            data_scientist_jobs[k[0]] = 1
    elif 'data' in k[0] and 'engineer' in k[0]:
        data_eng_des.append(k) 
        if k[0] in data_scientist_jobs.keys():
            data_scientist_jobs[k[0]] += 1
        else:
            data_scientist_jobs[k[0]] = 1
    elif 'data' in k[0] and 'analyst' in k[0]:
        data_ana_des.append(k) 
        if k[0] in data_scientist_jobs.keys():
            data_scientist_jobs[k[0]] += 1
        else:
            data_scientist_jobs[k[0]] = 1
    elif 'data scientist' in k[1]:
        datasci_related.append(k) 
        
    else:
        if k in  non_data_sc:
            continue
        else:
            non_data_sc.append(k) 
        print(k)
        


('applied scientist intern - alexa shopping', 'applied scientist intern - alexa shoppingamazon dev center (tel aviv)-תל אביב -יפו, מחוז תל אביבapply nowapply on company sitesave this jobתל אביב -יפו, מחוז תל אביב\nsecond year msc in computer science, or related field.\nsubmitted a research paper to a top conference.\nstrong cs foundations (data structures and algorithms).\nknowledge of programming languages such as c/c++, java, or python.\ngood writing and verbal english skills.\n\nyou: alexa, i am looking for a summer internship position where i could learn and innovate in ai, and impact the life of millions of people worldwide. it would be great if i could publish about it as well. what do you suggest?\nalexa: the alexa shopping team is looking for interns to help me become the best personal shopping assistant. do you want to hear more?\nyou: yes, please!\nalexa: as an intern in the alexa shopping research team, you will work with top researchers and engineers to explore and devise n

In [85]:
print('data science posting', len(data_sci_des))
print('data engineer posting', len(data_eng_des))
print('data analyst posting', len(data_ana_des))
print('datasci_related', len(datasci_related))
print('non data science', len(non_data_sc))


data science posting 96
data engineer posting 3
data analyst posting 4
datasci_related 45
non data science 11


In [86]:
for h in range(len(non_data_sc)):
    print(h)
    print(non_data_sc[h])
    print()

0
('applied scientist intern - alexa shopping', 'applied scientist intern - alexa shoppingamazon dev center (tel aviv)-תל אביב -יפו, מחוז תל אביבapply nowapply on company sitesave this jobתל אביב -יפו, מחוז תל אביב\nsecond year msc in computer science, or related field.\nsubmitted a research paper to a top conference.\nstrong cs foundations (data structures and algorithms).\nknowledge of programming languages such as c/c++, java, or python.\ngood writing and verbal english skills.\n\nyou: alexa, i am looking for a summer internship position where i could learn and innovate in ai, and impact the life of millions of people worldwide. it would be great if i could publish about it as well. what do you suggest?\nalexa: the alexa shopping team is looking for interns to help me become the best personal shopping assistant. do you want to hear more?\nyou: yes, please!\nalexa: as an intern in the alexa shopping research team, you will work with top researchers and engineers to explore and devise

In [83]:
# analyse sentenses in listing
sentences_list = []
sentence = re.compile('[^\n]+\n')
sentence_l = sentence.findall(non_data_sc[5][1])
print(sentence_l)

['applied scientist - alexa shoppingamazon.com-תל אביב -יפו, מחוז תל אביבapply nowapply on company sitesave this jobyou: alexa, i am looking for a new career opportunity, where i could conduct applied research, impact millions of customers, and publish about it in top conferences. what do you suggest?\n', 'alexa: the alexa shopping team is looking for brilliant applied researchers to help me become the best personal shopping assistant. do you want to hear more?\n', 'you: yes, please!\n', 'alexa: as an applied researcher in the alexa shopping research team, you will be responsible for research, design, and implementation of new ai technologies for voice assistants. you will collaborate with scientists, engineers, and product partners locally and abroad. your work will inventing, experimenting with, and launching new features, products and systems. ideally you have a expertise in at least one of the following fields: web search & data mining, machine learning, natural language processing

In [82]:
print(sentence_l)

['associate scientist/scientistcompugen usa-חולון, מחוז תל אביבapply nowapply on company sitesave this jobleading experimental target validation activities according to project wp and timelines. executing experiments relating to target validation activities (research tools generation and qc, expression studies, abs characterization, cell-based assays) from experimental design to final completion including documentation, reporting and presenting\n', 'experience and knowledge in protein purification, qc and analytics (column chromatography, hplc-sec, ms data interpretation, and sds-page)\n', 'highly experienced in cell culture technologies\n', '– over expression systems (lentivirus, retrovirus, transfections, etc.) including single clone generation and characterization, cell sorting\n', '– knock-down/knock out technologies (si\\shrna, crispr)\n', 'molecular biology methods\n', 'biochemical technologies\n', '– western blot, ip, elisa\n', 'cell based in vitro assays\n', '– experience in fa

In [ ]:
soup = BeautifulSoup(page.text, "html.parser")
#printing soup in a more structured tree format that makes for easier reading
print(soup.prettify())

In [ ]:
def extract_job_title_from_result(soup): 
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)
extract_job_title_from_result(soup)

In [ ]:
def extract_company_from_result(soup): 
    companies = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        company = div.find_all(name="span", attrs={"class":"company"})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
        else:
            sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
            for span in sec_try:
                companies.append(span.text.strip())
    return(companies)
 
extract_company_from_result(soup)


In [ ]:
def extract_location_from_result(soup): 
    locations = []
    spans = soup.findAll("span", attrs={'class': "location"})
    for span in spans:
        locations.append(span.text)
    return(locations)
extract_location_from_result(soup)

In [ ]:
def extract_summary_from_result(soup): 
    summaries = []
    spans = soup.findAll("span", attrs={"class": "summary"})
    for span in spans:
        summaries.append(span.text.strip())
    return(summaries)
extract_summary_from_result(soup)

In [ ]:
max_results_per_city = 25
#city_set = ["Tel+Aviv","Chicago", "New+York", "Boulder"]
city_set = ["Tel+Aviv"]
columns = ["city”, “job_title”, ”company_name”, ”location”, ”summary"]
#columns = ["city”, “job_title”, ”company_name”, ”summary"]
sample_df = pd.DataFrame(columns = columns)


In [ ]:
#scraping code:
for city in city_set:
    for start in range(0, max_results_per_city, 10):
        page = requests.get("https://il.indeed.com/jobs?q=data+scientist&l=tel+aviv&start=" + str(start))
        time.sleep(1)  #ensuring at least 1 second between page grabs
        soup = BeautifulSoup(page.text, "lxml", from_encoding="utf-8")
        for div in soup.find_all(name="div", attrs={"class":"row"}): 
            #specifying row num for index of job posting in dataframe
            #num = (len(sample_df) + 1)
            # think this shouldnt be +1
            num = (len(sample_df))
            #creating an empty list to hold the data for each posting
            job_post = [] 
            #append city name
            job_post.append(city) 
            #grabbing job title
            for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
   
                job_post.append(a["title"]) 
            #grabbing company name
            company = div.find_all(name="span", attrs={"class":"company"})
            if len(company) > 0:
                for b in company:
                    job_post.append(b.text.strip()) 
            else:
                sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
                for span in sec_try:
                    job_post.append(span.text) 
            print("this is row 1!!!!")
            print(job_post)
            #grabbing location name
          #  for div in soup.find_all(name="div", attrs={"class":"companyInfoWrapper"}): 
        
                
            c = soup.findAll("span", attrs={'class': "location"})
            for span in c:
                temp1 = 0
                temp1 = span.text
                job_post.append(span.text) 
                print()
                if len(temp1) > 0:
                    break
           
            #grabbing summary text
            d = soup.findAll("span", attrs={"class": "summary"})
            for span in d:
                temp2 = 0
                temp2 = span.text
                job_post.append(span.text.strip())
                if len(temp2) > 0:
                    break
           
            
            print("this is row 2!!!!")
            print(job_post)
            #appending list of job post info to dataframe at index num
            sample_df.iloc[num,:] = job_post
#saving sample_df as a local csv file — define your own local path to save contents 
#sample_df.to_csv("C:\Users\lili\Documents\icode\scraping\ver1.csv", encoding="utf-8")
print(sample_df.head())

In [ ]:
print(sample_df.head())

In [ ]:
import selenium


In [ ]:
page_link = 'https://il.indeed.com/viewjob?jk=34585bf90b5206ed&tk=1d3lup7jo9252800&from=serp&vjs=3&advn=9713374677006022&adid=224017829&sjdu=9nrDNPdV1DghkDNnC2WJlW1nrk21-asFvjSk9jx-s_NcPsvmgiL1SQSE_IEhrlP7Cc3Ep1zy7jW_xK_hDZJxGYPqhPkf11VoSbXetcsrKGaL4xu5qrLYw704UWhMz7qRBsy-sqTPfeze7xhSxmDXBe1BZZVW06BL8bXZ3NDTpxxtY2KNaxBsHYM8M4C6Q0ikHNqsYt1l-7GmHk9AaaZVJTUrXBDW0Mm858vBLFT9-SdDeee1d_uqkpa2KR2V1YfD4Nma5D7M5nHyTKG7LxVO8dqWl5prL9lDfhRSvM3490E-dPIu9LBxxKia8nDrgZixd3-bW1pDCT4Fkq1JnZY5iMBFZwECgBveLS2_W0q3CRd0PhOtQ1CycZ1T3SPEfEZsDItDMzK1NnWNrWG_0-IA6YuDZr1EGYbyUOhwHrzEZwk'

page_response = requests.get(page_link, timeout=5)
page_content = BeautifulSoup(page_response.content, "html.parser")
textContent = []
for i in range(0, 10):
    paragraphs = page_content.find_all("p")[i].text
    textContent.append(paragraphs)
    


In [ ]:
print( textContent)

# source
* [Web Scraping Job Postings from Indeed](https://medium.com/@msalmon00/web-scraping-job-postings-from-indeed-96bd588dcb4b)
* [Scraping Job Posting Data from Indeed using Selenium and BeautifulSoup](https://towardsdatascience.com/scraping-job-posting-data-from-indeed-using-selenium-and-beautifulsoup-dfc86230baac)
